本段開始進行BERTopic主題模型說明

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

os.chdir('/content/drive/MyDrive/SMA_final/') #切換該目錄
os.listdir() #確認目錄內容

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['chiikawa_data',
 'dict',
 'SMA_final_classification.ipynb',
 'chiikawa_unified_data.csv',
 'copy',
 'lda_visualization.html',
 'IG_coherence_scores.png',
 'lda_visualization_IG.html',
 'IG_topic_words.png',
 'YT_coherence_scores.png',
 'lda_visualization_YT.html',
 'YT_topic_words.png',
 'PTT_coherence_scores.png',
 'lda_visualization_PTT.html',
 'PTT_topic_words.png',
 'platform_comparison.png',
 'SMA_final_LDA.ipynb',
 'SMA_final_.ipynb',
 'gpt_api_key.txt']

In [1]:
# !pip install sentence_transformers
# !pip install bertopic

In [ ]:
import pandas as pd
import re
import numpy as np
from collections import defaultdict
import multiprocessing
import jieba
import matplotlib.pyplot as plt
from matplotlib.font_manager import fontManager
from sentence_transformers import SentenceTransformer

# 設定字體
fontManager.addfont('./copy/TaipeiSansTCBeta-Regular.ttf')
plt.rcParams['font.sans-serif'] = ['Taipei Sans TC Beta']
plt.rcParams['font.size'] = '16'

# Transformer 模型下的主題詮釋與文本關聯：以 BERT 嵌入為基礎的主題分析

使用 BERT 中文語意嵌入、HDBSCAN 分群與 BERTopic 主題建模技術，進行主題抽取與語意詮釋。最後，我們利用 GPT API 為每個主題生成描述。

---

## Step 1：資料讀取與清洗


In [ ]:
import os
import pandas as pd

# 讀取資料
df = pd.read_csv('chiikawa_unified_data.csv')


In [ ]:
# 對文本進行完整的清理
df['clean_content'] = df['artContent'].astype(str)
# 將多個連續換行符替換為句號
df['clean_content'] = df['clean_content'].str.replace(r'\n\n', '。', regex=True)
# 將單個換行符替換為逗號
df['clean_content'] = df['clean_content'].str.replace(r'\n', '，', regex=True)
# 移除網址
df['clean_content'] = df['clean_content'].str.replace(r'http\S+', '', regex=True).str.replace(r'www\S+', '', regex=True)
# 統一標點符號，移除表情符號和特殊符號
df['clean_content'] = df['clean_content'].str.replace(r'[^\u4e00-\u9fff\s.,!?;:、，。！？；：]', '', regex=True)

# 使用正則表達式進行更精確的斷句
def split_sentences(text):
    # 以句號、問號、感嘆號等作為斷句標記
    sentences = re.split(r'[。！？!?]+', text)
    # 移除空字符串和只包含空白的字符串
    return [s.strip() for s in sentences if s.strip()]

df['sentences'] = df['clean_content'].apply(split_sentences)

# 檢視斷句結果
sample_sentences = df[df['sentences'].str.len() > 0].iloc[0]['sentences']
print(f"斷句範例：{sample_sentences[:3]}")  # 顯示前3個句子

斷句範例：['小八跳舞 吉伊卡哇']


In [ ]:
# 設定jieba的字典和使用者自定義詞典
jieba.set_dictionary('dict/dict.txt.big')  # 使用繁體中文字典

# 新增特定領域詞彙到自定義詞典
custom_words = [
    # 主要名稱
    "Chiikawa", "吉伊卡哇", "ちいかわ", "吉伊", "小可愛", "吉依", "寶寶",

    # 主要角色
    "小八貓", "ハチワレ", "Hachiware", "八字瀏海", "小八", "藍色褲頭貓", "哈奇" , "哈吉",
    "兔兔", "うさぎ", "Usagi", "烏薩奇", "兔哥", "537",
    "小桃", "モモンガ", "Momonga", "飛鼠",
    "海獺勇者", "ラッコ", "Rakko",
    "栗子饅頭", "くりまんじゅう", "Kuri-Manjuu", "前輩",

    # 其他角色
    "風獅", "シーサー", "Shisa", "獅薩",
    "拉麵鎧甲人", "手拿包鎧甲人", "勞動鎧甲人", "鎧甲人",
    "睡衣派對", "パジャマパーティーズ", "哥布林",

    # 特殊詞彙
    "吉伊卡哇構文", "除草考試", "除草檢定", "小可愛族",
    "郎拉麵店", "三級除草證件", "討伐", "抽獎運",

    # 特性描述
    "膽小鬼", "愛哭鬼", "療癒", "二頭身", "啊哈", "吖哈",
    "又小又可愛", "なんか小さくてかわいいやつ",

    # 創作相關
    "Nagano", "ナガノ", "動畫工房", "周邊商品", "貼圖", "一番賞"
]

for word in custom_words:
    jieba.add_word(word, freq=100)  # 給予較高詞頻以確保能被切分出來

# 建立角色同義詞映射表
character_synonyms = {
    # 主角同義詞映射
    "ちいかわ": "吉伊卡哇", "Chiikawa": "吉伊卡哇", "吉伊": "吉伊卡哇",
    "小可愛": "吉伊卡哇", "吉依": "吉伊卡哇",

    # 小八貓同義詞映射
    "ハチワレ": "小八貓", "Hachiware": "小八貓", "八字瀏海": "小八貓",
    "小八": "小八貓", "藍色褲頭貓": "小八貓", "哈奇": "小八貓", "哈吉": "小八貓",

    # 兔兔同義詞映射
    "うさぎ": "兔兔", "Usagi": "兔兔", "烏薩奇": "兔兔",
    "兔哥": "兔兔", "537": "兔兔",

    # 其他角色同義詞映射
    "モモンガ": "小桃", "Momonga": "小桃", "飛鼠": "小桃",
    "ラッコ": "海獺勇者", "Rakko": "海獺勇者",
    "くりまんじゅう": "栗子饅頭", "Kuri-Manjuu": "栗子饅頭", "前輩": "栗子饅頭",
    "シーサー": "風獅", "Shisa": "風獅", "獅薩": "風獅",
    "パジャマパーティーズ": "睡衣派對", "哥布林": "睡衣派對"
}

# 載入停用詞
def load_stopwords(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            stopwords = [line.strip() for line in f.readlines()]
        return set(stopwords)
    except UnicodeDecodeError:
        with open(file_path, 'r', encoding='big5') as f:
            stopwords = [line.strip() for line in f.readlines()]
        return set(stopwords)
    except Exception as e:
        print(f"讀取停用詞檔案時發生錯誤: {e}")
        return set()

# 讀取停用詞
stopwords_path = "./dict/stopwords.txt"
stopwords = load_stopwords(stopwords_path)
print(f"已載入 {len(stopwords)} 個停用詞")

# 新增自定義停用詞
custom_stopwords = [
    # 代詞
    "我", "你", "他", "她", "它", "我們", "你們", "他們", "她們", "它們", "這", "那", "這些", "那些", "誰", "什麼", "哪", "哪裡", "哪兒", "怎麼", "怎樣", "如何",

    # 連接詞
    "和", "與", "而", "並", "或", "但", "但是", "然而", "所以", "因為", "因此", "如果", "雖然", "即使", "無論", "只要", "不論", "假如", "若", "若是",

    # 助詞
    "的", "地", "得", "了", "著", "過", "吧", "嗎", "呢", "啊", "哦", "喔", "呀", "耶", "哎", "唉", "嗯", "嘿",

    # 副詞
    "很", "非常", "極", "太", "更", "最", "又", "也", "都", "還", "只", "就", "才", "剛", "曾", "已", "將", "不", "沒", "別", "莫", "勿",

    # 介詞
    "在", "從", "向", "往", "於", "對", "給", "為", "替", "由", "把", "被", "讓", "使", "隨", "跟", "靠", "據", "至", "到", "自",

    # 數量詞
    "一", "二", "三", "四", "五", "六", "七", "八", "九", "十", "百", "千", "萬", "億", "兆", "個", "些", "每", "某", "各", "整", "全", "多", "少", "幾",

    # 時間詞
    "年", "月", "日", "時", "分", "秒", "天", "週", "季", "今", "昨", "明", "前", "後", "當", "正", "現", "曾", "已", "將", "會", "常", "永",

    # 方位詞
    "上", "下", "左", "右", "前", "後", "內", "外", "中", "東", "西", "南", "北", "中間", "旁邊", "附近", "周圍",

    # 常用動詞
    "是", "有", "無", "來", "去", "做", "看", "聽", "說", "想", "要", "能", "會", "可以", "應該", "必須", "需要", "得", "獲", "取", "予",

    # 常用形容詞
    "好", "壞", "大", "小", "多", "少", "高", "低", "長", "短", "新", "舊", "快", "慢", "遠", "近", "輕", "重", "深", "淺",

    # 其他常見虛詞
    "之", "乎", "以", "其", "所", "者", "矣", "焉", "哉", "也", "然", "否", "兮", "乃", "且", "為", "則", "再", "又", "此", "夫", "亦",

    # 常見短語
    "一個", "這個", "那個", "一些", "這些", "那些", "一下", "一點", "一直", "一定", "一般", "一樣", "不過", "不必", "不要", "可能", "可是", "只是", "就是", "如此", "如果", "這樣", "那樣", "這麼", "那麼", "因此", "所以", "然後", "接著", "其實", "其中", "其他", "其它", "之前", "之後", "之中", "之間",

    # 網路用語
    "http", "www", "com", "cn", "org", "net", "html", "htm", "php", "jpg", "png", "gif", "..", "\r"
]
stopwords.update(custom_stopwords)



Building prefix dict from /content/drive/.shortcut-targets-by-id/19AP0P8Z6LMG3msxuCawQ0Nqe0na8M-Zq/SMA_final/dict/dict.txt.big ...
DEBUG:jieba:Building prefix dict from /content/drive/.shortcut-targets-by-id/19AP0P8Z6LMG3msxuCawQ0Nqe0na8M-Zq/SMA_final/dict/dict.txt.big ...
Loading model from cache /tmp/jieba.uf17e4a7ef325f0706c3d60a3b587a234.cache
DEBUG:jieba:Loading model from cache /tmp/jieba.uf17e4a7ef325f0706c3d60a3b587a234.cache
Loading model cost 1.429 seconds.
DEBUG:jieba:Loading model cost 1.429 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


已載入 1211 個停用詞


In [ ]:
def preprocess(text, stopwords):
    # 使用 jieba 分詞
    words = jieba.cut(text)
    # 移除停用詞與空字串
    filtered_words = [w for w in words if w.strip() and w not in stopwords]
    return " ".join(filtered_words)  # 用空白分隔，方便 CountVectorizer 使用

# 套用到 df["clean_content"]
df["processed_text"] = df["clean_content"].apply(lambda x: preprocess(x, stopwords))

In [ ]:
df.head()

,system_id,dataSource,artUrl,artDate,artTitle,artContent,artPoster,artPosterUrl,commentcount,likecount,category,clean_content,sentences,processed_text
0,1,YT,https://www.youtube.com/watch?v=3fmZbTHG8us,2024-07-06 18:46:55,小八跳舞 #吉伊卡哇,小八跳舞 #吉伊卡哇,軒仔,https://www.youtube.com/channel/UCXYcyfr619aUF...,0.0,4.0,NaN,小八跳舞 吉伊卡哇,[小八跳舞 吉伊卡哇],小八 跳舞 吉伊卡哇
1,2,YT,https://www.youtube.com/watch?v=kcilIzr7Bf0,2024-07-07 04:48:58,【Chiikawa】睡衣派對之歌 / パジャマパーティーズのうた 廣東話歌詞,【Chiikawa】睡衣派對之歌 / パジャマパーティーズのうた 廣東話歌詞你好，禾是幼女妹...,MK妹 Ch.,https://www.youtube.com/channel/UCO62chyehk6pX...,1.0,1.0,NaN,睡衣派對之歌 廣東話歌詞你好，禾是幼女妹，，因為妹條婆奴役禾跳舞，導致我依家手腳不協調入...,[睡衣派對之歌 廣東話歌詞你好，禾是幼女妹，，因為妹條婆奴役禾跳舞，導致我依家手腳不協調...,睡衣派對 之歌 廣東話 歌詞 你好 禾 幼女 妹 因為妹 條婆 奴役 禾 跳舞 導致 依家 ...
2,3,YT,https://www.youtube.com/watch?v=6EtTJFbRRaE,2024-07-08 07:07:35,【🔴Live直播】開箱華山快閃店！「史努比、吉伊卡哇」萌翻,【🔴Live直播】開箱華山快閃店！「史努比、吉伊卡哇」萌翻暑假到了，想外出又怕天氣太熱？快到...,年代新聞CH50,https://www.youtube.com/channel/UCHBv5vSp3pETS...,0.0,1116.0,NaN,直播開箱華山快閃店！史努比、吉伊卡哇萌翻暑假到了，想外出又怕天氣太熱？快到華山逛快閃店吹免費...,"[直播開箱華山快閃店, 史努比、吉伊卡哇萌翻暑假到了，想外出又怕天氣太熱, 快到華山逛快閃店...",直播 開箱 華山 閃店 史努比 吉伊卡哇 萌翻 暑假 外出 天氣 太熱 快到 華山 逛 閃店...
3,4,YT,https://www.youtube.com/watch?v=r4ZtjfGZQaA,2024-07-15 03:07:46,【突發LA】迫不及待的盲抽🌟拆熱騰騰剛到的吉伊卡哇卡牌盲包🐺 #森森鈴蘭,【突發LA】迫不及待的盲抽🌟拆熱騰騰剛到的吉伊卡哇卡牌盲包🐺 #森森鈴蘭#歌唱系 #森森鈴蘭...,森森鈴蘭 / Lily Linglan【箱箱The Box所屬】,https://www.youtube.com/channel/UC3ZTQ8VZVCpwL...,27.0,7601.0,NaN,突發迫不及待的盲抽拆熱騰騰剛到的吉伊卡哇卡牌盲包 森森鈴蘭歌唱系 森森鈴蘭 ，迫不及待的盲...,[突發迫不及待的盲抽拆熱騰騰剛到的吉伊卡哇卡牌盲包 森森鈴蘭歌唱系 森森鈴蘭 ，迫不及待的...,突發 迫不及待 盲 抽 拆 熱騰騰 剛到 吉伊卡哇 卡牌 盲包 森森 鈴蘭 歌唱 系 森森 ...
4,5,YT,https://www.youtube.com/watch?v=uFynuUT3mHI,2024-07-16 14:27:08,【戰利品開箱LA】笑死 買到沒錢💸 #森森鈴蘭 #吉伊卡哇 #我要記得宣傳演唱會RRR,【戰利品開箱LA】笑死 買到沒錢💸 #森森鈴蘭 #吉伊卡哇 #我要記得宣傳演唱會RRR#歌唱...,森森鈴蘭 / Lily Linglan【箱箱The Box所屬】,https://www.youtube.com/channel/UC3ZTQ8VZVCpwL...,20.0,9154.0,NaN,戰利品開箱笑死 買到沒錢 森森鈴蘭 吉伊卡哇 我要記得宣傳演唱會歌唱系 森森鈴蘭 ，回煉金...,[戰利品開箱笑死 買到沒錢 森森鈴蘭 吉伊卡哇 我要記得宣傳演唱會歌唱系 森森鈴蘭 ，回煉...,戰利品 開箱 笑 死 買到 沒錢 森森 鈴蘭 吉伊卡哇 我要 記得 宣傳 演唱會 歌唱 系 ...


In [ ]:
# 假設「artContent」是文章內容，「category」是標籤
df = df.rename(columns={'processed_text': 'text', 'category': 'label'})

# 移除缺失值
df = df.dropna(subset=['text', 'label'])

## Step 2：建立繁體中文嵌入與分詞工具
使用 `google-bert/bert-base-chinese` 作為語意嵌入模型，並搭配 `jieba` 進行繁體中文斷詞


In [ ]:
# 套用到每篇文章
docs_zh = df['text'].tolist()
docs_zh = [doc for doc in docs_zh if doc.strip()]

# === Step 3: BERT 中文嵌入 ===
bert_sentence_model = SentenceTransformer("google-bert/bert-base-chinese")
embeddings = bert_sentence_model.encode(docs_zh, show_progress_bar=True)

Batches:   0%|          | 0/375 [00:00<?, ?it/s]

## Step 3：BERTopic 主題模型訓練（含自訂 jieba 分詞器）
我們使用 HDBSCAN 進行主題分群，並以 TF 向量從每群中擷取代表性詞彙

In [ ]:
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic


hdbscan_model = HDBSCAN(
    min_cluster_size=50,
    min_samples=5,
    metric='euclidean',
    cluster_selection_method='leaf'
)
# 中文分詞函數
def tokenize_zh(text):
    return jieba.lcut(text)

# 使用 jieba 分詞的 CountVectorizer
jieba_vectorizer = CountVectorizer(
    tokenizer=tokenize_zh,
    stop_words=list(stopwords),
    analyzer='word',
    token_pattern=u"(?u)\\b\\w+\\b"
)

# 建立 BERTopic 模型
zh_topic_model = BERTopic(
    embedding_model=bert_sentence_model,
    vectorizer_model=jieba_vectorizer,
    hdbscan_model=hdbscan_model,
    verbose=True,
    top_n_words=30
)

# 執行模型訓練
topics, probs = zh_topic_model.fit_transform(docs_zh, embeddings)


2025-06-09 12:48:02,062 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-06-09 12:48:15,472 - BERTopic - Dimensionality - Completed ✓
2025-06-09 12:48:15,473 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-06-09 12:48:15,879 - BERTopic - Cluster - Completed ✓
2025-06-09 12:48:15,887 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-06-09 12:48:23,822 - BERTopic - Representation - Completed ✓


## Step 4：主題檢視與關鍵詞提取
查看每個主題的代表性關鍵詞與文章數

In [ ]:
zh_topic_model.reduce_topics(docs_zh, nr_topics=20)
zh_topic_model.get_topic_info().head(10)

2025-06-09 12:49:58,036 - BERTopic - Topic reduction - Reducing number of topics
2025-06-09 12:49:58,186 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-06-09 12:50:07,161 - BERTopic - Representation - Completed ✓
2025-06-09 12:50:07,165 - BERTopic - Topic reduction - Reduced number of topics from 75 to 20


,Topic,Count,Name,Representation,Representative_Docs
0,-1,4655,-1_ _吉伊卡哇_日本_銅鑼灣,"[ , 吉伊卡哇, 日本, 銅鑼灣, 公仔, 玩具, 代購, 可愛, 買, 嘅, 香港, 小...",[多款 公仔 冇 發現 小可愛 嘅 設計 超靚 公仔 公仔 代購 吉伊卡哇 吉伊卡哇 小可愛...
1,0,2105,0_ _吉伊卡哇_可愛_壽司,"[ , 吉伊卡哇, 可愛, 壽司, 活動, 日本, 商品, 美甲, 藏, 聯名, 高雄, 限...",[美食 藏 壽司 推出 吉伊卡哇 扭蛋 日本 人氣 吉伊卡哇 藏 壽司 首次 合作 一口氣 ...
2,1,1201,1_ _代購_公仔_小可愛,"[ , 代購, 公仔, 小可愛, 門市, 八字, 旺角, 貓, 付款, 日本, 查詢, 地址...",[代購 小可愛 壽司 系列 公仔 掛飾 現在 代購 有意 網店 下單 下單 傳送 售完 即止...
3,2,953,2_銅鑼灣_玩具_ _日本,"[銅鑼灣, 玩具, , 日本, 時代廣場, 購物, 電視, 百貨, 精品, 對面, 公仔,...",[公仔 代購 吉伊卡哇 吉伊卡哇 小可愛 吉伊卡哇 代購 吉伊卡哇 公仔 銅鑼灣 時代廣場 ...
4,3,475,3_迷因_追蹤_喜歡_翻譯,"[迷因, 追蹤, 喜歡, 翻譯, 阿甘妙, 梗圖, 梗, 幽默, 搞笑, 人生, 圖, 世界...",[經典 喜歡 吉伊卡哇 追蹤 喜歡 阿甘妙 世界 追蹤 喜歡 酷洛米 追蹤 迷因 迷因 梗 ...
5,4,387,4_ _吉伊卡哇_吉伊_可愛,"[ , 吉伊卡哇, 吉伊, 可愛, 烏薩奇, 小八, 知道, 題, 真的, 最近, 動畫, ...",[台 有用 原創 看不出來 有人 要說 原創 紅的 多弄個 白爛貓 不錯 弄 這三小 引述 ...
6,5,279,5_烏薩奇_代購_小八貓_吉伊卡哇,"[烏薩奇, 代購, 小八貓, 吉伊卡哇, 公仔, , 日本, 小可愛, 小八, 兔哥, 吉...","[代購 小可愛 小八貓 烏薩奇 吉伊卡哇 吉伊卡哇 吉伊卡哇 代購, 小可愛 烏薩奇 公仔 ..."
7,6,255,6_又小又可愛_吉伊卡哇_ _福井,"[又小又可愛, 吉伊卡哇, , 福井, 太太, 版, 可愛, 蘋, 小可愛, , , , ...","[吉伊卡哇 又小又可愛, 吉伊卡哇 又小又可愛, 吉伊卡哇 又小又可愛]"
8,7,216,7_ _網站_係_嘅,"[ , 網站, 係, 嘅, 貨品, 唔, 睇, 佢, 下單, 日本, 支持, 自助, 咗, ...",[日本 預訂 得意 圖案 新出 款個 味道 睇 佢 得意 樣 覺得 成枝 網站 每日 新款 ...
9,8,179,8_留言_連結_ _底下,"[留言, 連結, , 底下, 給你, 吉伊卡哇, 可愛, 唷, 傳送, 真的, 超, 限定...",[熱騰騰 實拍 周邊 日本 搶手 款 通通 元 價 吉伊卡哇 坐車 車會 可愛 出遊 系列 ...


互動式圖表探索主題空間與主題關鍵詞

In [ ]:
zh_topic_model.visualize_topics()

## Step 5：使用 GPT 為主題自動生成描述
我們將每個主題的前 10 個關鍵詞輸入 GPT，讓它推測此群主題所屬分類描述

In [2]:
# !pip install openai

In [ ]:
# 讀取API key
with open("gpt_api_key.txt", "r", encoding="utf-8") as f:
    key = f.read().strip()

In [ ]:
from openai import OpenAI
client = OpenAI(api_key=key)
def call_gpt(keywords):
    try:
        prompt = f"""你是一位主題詮釋專家。請根據以下關鍵詞推論它們所代表的主題，並以繁體中文簡要描述該主題的內容。

關鍵詞：{", ".join(keywords)}

請只回覆主題敘述，不需要列出主題類別，也不要加任何額外說明。"""
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.8
        )
        output = response.choices[0].message.content.strip()
        return output
    except Exception as e:
        print(f"⚠️ Error: {e}")
        return ""

In [ ]:
import openai
import time

# 擷取主題關鍵詞
topic_keywords = {
    topic: [w[0] for w in zh_topic_model.get_topic(topic)[:10]]
    for topic in zh_topic_model.get_topics().keys()
    if topic != -1
}

topic_keywords


{0: [' ', '吉伊卡哇', '可愛', '壽司', '活動', '日本', '商品', '美甲', '藏', '聯名'],
 1: [' ', '代購', '公仔', '小可愛', '門市', '八字', '旺角', '貓', '付款', '日本'],
 2: ['銅鑼灣', '玩具', ' ', '日本', '時代廣場', '購物', '電視', '百貨', '精品', '對面'],
 3: ['迷因', '追蹤', '喜歡', '翻譯', '阿甘妙', '梗圖', '梗', '幽默', '搞笑', '人生'],
 4: [' ', '吉伊卡哇', '吉伊', '可愛', '烏薩奇', '小八', '知道', '題', '真的', '最近'],
 5: ['烏薩奇', '代購', '小八貓', '吉伊卡哇', '公仔', ' ', '日本', '小可愛', '小八', '兔哥'],
 6: ['又小又可愛', '吉伊卡哇', ' ', '福井', '太太', '版', '可愛', '蘋', '小可愛', ''],
 7: [' ', '網站', '係', '嘅', '貨品', '唔', '睇', '佢', '下單', '日本'],
 8: ['留言', '連結', ' ', '底下', '給你', '吉伊卡哇', '可愛', '唷', '傳送', '真的'],
 9: ['烏薩奇', '栗', '小八', '栗子饅頭', '吉伊卡哇', '卡哇', '吉依', '兔哥', '海獺', '小八貓'],
 10: ['蛋糕', '客', ' ', '製', '蛋糕店', '化', '生日蛋糕', '訂購', '百日', '訂做'],
 11: ['撮', '写', '録', '影友会', '活記', '好人', '繋', '攝影', '小可愛', '生活'],
 12: ['唔', '係', ' ', '咗', '咁', '真', '買', '冇', '睇', '知'],
 13: ['绝区', '吉伊卡哇', '布', '抓', '哥布林', '服装', '金刚', '与', '狼', '壁纸'],
 14: ['吉伊卡哇', '網址', '吉伊', '預訂', '嗚嗚', '哇哇', '吉', '口哇', ' ', '吉其'],
 15: ['露天', '

In [ ]:
# 執行 GPT 詮釋
gpt_labels = {}
for topic_id, keywords in topic_keywords.items():
    print(f"\n 主題 {topic_id} 關鍵字: {', '.join(keywords)}")
    label = call_gpt(keywords)
    print(f"GPT 推論：{label}")
    time.sleep(1.5)



 主題 0 關鍵字:  , 吉伊卡哇, 可愛, 壽司, 活動, 日本, 商品, 美甲, 藏, 聯名
GPT 推論：這個主題圍繞著日本流行文化中的可愛元素，特別是吉伊卡哇（Kawaii）風格，將可愛的角色和壽司主題結合，推出各種商品，如美甲藝術和聯名商品。這些商品通常會在特定的活動中展示，吸引喜愛可愛文化的消費者，並且可能隱含著收藏的樂趣。

 主題 1 關鍵字:  , 代購, 公仔, 小可愛, 門市, 八字, 旺角, 貓, 付款, 日本
GPT 推論：這個主題圍繞著在旺角地區的代購服務，主要針對日本的公仔和小可愛商品。消費者可以透過代購平台購買心儀的貓咪相關商品，並在門市完成付款，享受便捷的購物體驗。這些商品通常具有獨特的設計和高人氣，滿足喜愛日本文化和收藏的需求。

 主題 2 關鍵字: 銅鑼灣, 玩具,  , 日本, 時代廣場, 購物, 電視, 百貨, 精品, 對面
GPT 推論：這個主題圍繞銅鑼灣的購物體驗，特別是以玩具和日本商品為主的時代廣場以及周邊的百貨和精品店。銅鑼灣作為一個繁華的商業區，吸引了眾多遊客和當地居民來此購物，享受多樣化的商品選擇，並且在店鋪內外的電視屏幕上可以看到最新的促銷和產品介紹，增添了購物的樂趣和氛圍。對面的商店則提供了更多選擇，形成了一個熱鬧的購物環境。

 主題 3 關鍵字: 迷因, 追蹤, 喜歡, 翻譯, 阿甘妙, 梗圖, 梗, 幽默, 搞笑, 人生
GPT 推論：這個主題圍繞著網路文化中的迷因和梗圖，探討它們如何透過幽默和搞笑的方式反映人生的各種現象。迷因常常被翻譯和追蹤，讓更多人可以分享和喜歡這些創作，並在社交媒體上形成共鳴。阿甘妙等人物或情境成為梗的來源，透過幽默的方式揭示生活中的諷刺與真實，引發人們的思考與共鳴。

 主題 4 關鍵字:  , 吉伊卡哇, 吉伊, 可愛, 烏薩奇, 小八, 知道, 題, 真的, 最近
GPT 推論：這個主題主要圍繞著吉伊卡哇（Gigi）這個可愛角色的相關內容，可能涉及她的形象與受歡迎程度。最近的討論中，人們對於吉伊卡哇及其搭檔烏薩奇（Usagi）和小八（Shiba Inu）等角色的喜愛與評價，表現出對這些角色的深厚情感與趣味。在社交媒體或相關平台上，網友們分享了他們對這些角色的見解與趣聞，顯示出一種對可愛文化的熱潮。

 主題 5 關鍵字: 烏薩奇, 代購, 小八貓, 吉伊卡哇, 公仔,